<a href="https://colab.research.google.com/github/vincimech010233/QuantumComputingJourney-/blob/main/Iris_pennylane.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pennylane scikit-learn torch

In [3]:
import numpy as np
import torch
import torch.nn as nn
import pennylane as qml
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [4]:
# Cargar el dataset de Iris
data = load_iris()
X, y = data.data, data.target

In [5]:
# Simplificar el problema a una clasificación binaria
X = X[:100]  # Usamos solo las primeras 100 muestras para Setosa y Versicolor
y = y[:100]

In [6]:
# Preprocesar los datos
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [7]:
# Convertir a tensores de PyTorch
X = torch.tensor(X, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.float32).view(-1, 1)

In [8]:
# Dividir los datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [9]:
# Configurar el dispositivo cuántico
dev = qml.device('default.qubit', wires=1)

In [10]:
# Definir el circuito cuántico
def circuit(params, x):
    qml.Rot(x[0], x[1], x[2], wires=0)
    qml.RY(params[0], wires=0)
    return qml.expval(qml.PauliZ(0))

In [13]:
# Crear el modelo híbrido
class HybridModel(nn.Module):
    def __init__(self):
        super(HybridModel, self).__init__()
        self.pre_net = nn.Linear(4, 3)
        self.q_params = nn.Parameter(torch.rand(1))
        self.qnode = qml.QNode(circuit, dev, interface='torch')
        self.post_net = nn.Linear(1, 1)

    def forward(self, x):
        x = self.pre_net(x)
        x = torch.tanh(x)
        x = [self.qnode(self.q_params, x_) for x_ in x]
        x = torch.tensor(x).float().unsqueeze(-1)
        x = self.post_net(x)
        return torch.sigmoid(x)

model = HybridModel()

In [14]:
# Entrenar el modelo
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = nn.BCELoss()

In [15]:
epochs = 50
for epoch in range(epochs):
    optimizer.zero_grad()
    y_pred = model(X_train)
    loss = criterion(y_pred, y_train)
    loss.backward()
    optimizer.step()
    if epoch % 5 == 0:
        print(f"Epoch {epoch}, Loss: {loss.item()}")

Epoch 0, Loss: 0.6954945921897888
Epoch 5, Loss: 0.6856858134269714
Epoch 10, Loss: 0.6779828071594238
Epoch 15, Loss: 0.6722280979156494
Epoch 20, Loss: 0.667880654335022
Epoch 25, Loss: 0.6641548871994019
Epoch 30, Loss: 0.6604390740394592
Epoch 35, Loss: 0.6565159559249878
Epoch 40, Loss: 0.6524172425270081
Epoch 45, Loss: 0.6482306718826294


In [16]:
# Evaluar el modelo en el conjunto de prueba
with torch.no_grad():
    y_pred = model(X_test)
    predictions = y_pred.round()
    accuracy = (predictions == y_test).float().mean()
    print(f"Accuracy: {accuracy.item()}")

Accuracy: 0.44999998807907104
